In [64]:
# ---------- Imports ---------- #
from pydantic import BaseModel, Field
from typing import List, Dict
import urllib.request
import itertools
import json

In [65]:
# ---------- Request and response models ---------- #
class OrderDetails(BaseModel):
    garmentType: str = Field(..., examples=["Shirt", "Pants"])
    material: str = Field(..., examples=["Cotton", "Polyester"])
    unitWeight: float = Field(..., gt=0, examples=[0.30, 1.20])
    size: str = Field(..., examples=["XS", "S", "M", "L", "XL"])
    collection: str = Field(..., examples=["Summer", "Winter"])

class PackagingConfigurations(BaseModel):
    unitsPerPackage: int = Field(..., gt=0, examples=[1, 5, 10])
    foldingMethod: str = Field(..., examples=["Method1", "Method2"])
    layout: str = Field(..., examples=["LayoutA", "LayoutB"])
    supplier: str = Field(..., examples=["SupplierA", "SupplierB"])


In [ ]:
# ---------- Helper functions ---------- #
def build_request_data(orderDetails: OrderDetails):
    COLUMNS = [
        "SupplierName", "GarmentType", "Material", "Weight",
        "ProposedUnitsPerCarton", "ProposedFoldingMethod",
        "ProposedLayout", "Size", "Collection"
    ]

    GARMENT_TYPE = orderDetails.garmentType
    MATERIAL = orderDetails.material
    WEIGHT = orderDetails.unitWeight
    SIZE = orderDetails.size
    COLLECTION = orderDetails.collection


    FOLDING_METHODS = ["Method1", "Method2", "Method3"]
    LAYOUTS = ["LayoutA", "LayoutB", "LayoutC", "LayoutD", "LayoutE"]
    UNITS_RANGE = list(range(5, 50, 5))
    SUPPLIERS = [
        "SupplierA", "SupplierB", "SupplierC", "SupplierD",
        "SupplierE", "SupplierF", "SupplierG", "SupplierH"
    ]

    data_rows = [
        [
            supplier,
            GARMENT_TYPE,
            MATERIAL,
            WEIGHT,
            units,
            folding,
            layout,
            SIZE,
            COLLECTION,
        ]
        for supplier, units, folding, layout in itertools.product(
            SUPPLIERS,
            UNITS_RANGE,
            FOLDING_METHODS,
            LAYOUTS,
        )
    ]

    request_data = {
        "input_data": {
            "index": [0, 1],
            "columns": COLUMNS,
            "data": data_rows,
        }
    }

    return request_data



def execute_request(body):
    url = 'https://pckg-quality-endpoint.eastus.inference.ml.azure.com/score'
    
    api_key = ''
    if not api_key:
        raise Exception("A key should be provided to invoke the endpoint")


    headers = {'Content-Type':'application/json', 'Accept': 'application/json', 'Authorization':('Bearer '+ api_key)}

    req = urllib.request.Request(url, body, headers)

    try:
        response = urllib.request.urlopen(req)

        return response
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))

        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))

def to_packaging_config(packaging_detail: List) -> PackagingConfigurations:
    supplier, _, _, _, units, folding, layout, *_ = packaging_detail
    return PackagingConfigurations(
        unitsPerPackage=units,
        foldingMethod=folding,
        layout=layout,
        supplier=supplier
    )


def build_configurations(request_data, result) -> List[PackagingConfigurations]:
    packaging_details = request_data["input_data"]["data"]

    packaging_details_enriched = [order + [score]
                for order, score in zip(packaging_details, result)]
        
    best_overall = max(packaging_details_enriched, key=lambda r: r[-1])

    best_per_supplier: Dict[str, List] = {}
    for detail in packaging_details_enriched:
        supplier, *_, score = detail
        if supplier not in best_per_supplier or score > best_per_supplier[supplier][-1]:
            best_per_supplier[supplier] = detail

    configs: List[PackagingConfigurations] = []
    configs.append(to_packaging_config(best_overall))

    for entry in best_per_supplier.values():
        configs.append(to_packaging_config(entry))

    return configs

In [57]:
data = OrderDetails(
    garmentType="Shirt",
    material="Polyester",
    unitWeight=0.16,
    size="L",
    collection="Summer"
)

In [58]:
request_data = build_request_data(data)
request_data

{'input_data': {'index': [0, 1],
  'columns': ['SupplierName',
   'GarmentType',
   'Material',
   'Weight',
   'ProposedUnitsPerCarton',
   'ProposedFoldingMethod',
   'ProposedLayout',
   'Size',
   'Collection'],
  'data': [['SupplierA',
    'Shirt',
    'Polyester',
    0.16,
    5,
    'Method1',
    'LayoutA',
    'L',
    'Summer'],
   ['SupplierA',
    'Shirt',
    'Polyester',
    0.16,
    5,
    'Method1',
    'LayoutB',
    'L',
    'Summer'],
   ['SupplierA',
    'Shirt',
    'Polyester',
    0.16,
    5,
    'Method1',
    'LayoutC',
    'L',
    'Summer'],
   ['SupplierA',
    'Shirt',
    'Polyester',
    0.16,
    5,
    'Method1',
    'LayoutD',
    'L',
    'Summer'],
   ['SupplierA',
    'Shirt',
    'Polyester',
    0.16,
    5,
    'Method1',
    'LayoutE',
    'L',
    'Summer'],
   ['SupplierA',
    'Shirt',
    'Polyester',
    0.16,
    5,
    'Method2',
    'LayoutA',
    'L',
    'Summer'],
   ['SupplierA',
    'Shirt',
    'Polyester',
    0.16,
    5,
    

In [60]:
body = str.encode(json.dumps(request_data))
body

b'{"input_data": {"index": [0, 1], "columns": ["SupplierName", "GarmentType", "Material", "Weight", "ProposedUnitsPerCarton", "ProposedFoldingMethod", "ProposedLayout", "Size", "Collection"], "data": [["SupplierA", "Shirt", "Polyester", 0.16, 5, "Method1", "LayoutA", "L", "Summer"], ["SupplierA", "Shirt", "Polyester", 0.16, 5, "Method1", "LayoutB", "L", "Summer"], ["SupplierA", "Shirt", "Polyester", 0.16, 5, "Method1", "LayoutC", "L", "Summer"], ["SupplierA", "Shirt", "Polyester", 0.16, 5, "Method1", "LayoutD", "L", "Summer"], ["SupplierA", "Shirt", "Polyester", 0.16, 5, "Method1", "LayoutE", "L", "Summer"], ["SupplierA", "Shirt", "Polyester", 0.16, 5, "Method2", "LayoutA", "L", "Summer"], ["SupplierA", "Shirt", "Polyester", 0.16, 5, "Method2", "LayoutB", "L", "Summer"], ["SupplierA", "Shirt", "Polyester", 0.16, 5, "Method2", "LayoutC", "L", "Summer"], ["SupplierA", "Shirt", "Polyester", 0.16, 5, "Method2", "LayoutD", "L", "Summer"], ["SupplierA", "Shirt", "Polyester", 0.16, 5, "Method

In [61]:
response = execute_request(body)
response

In [62]:
result = json.loads(response.read())
result

[0.8531564849842795,
 0.8527787105607668,
 0.8957827639291442,
 0.8771591439314597,
 0.832663101455652,
 0.8755177802540264,
 0.8816778069363652,
 0.9353826409854317,
 0.9192738753959642,
 0.8649934597812511,
 0.8816256505135615,
 0.8794148685868287,
 0.9167937751907068,
 0.901512011115606,
 0.8615546363353995,
 0.8445801493558043,
 0.844184340771244,
 0.8914998638967528,
 0.872220064478448,
 0.8050867882201559,
 0.8762215418335235,
 0.8823514224869108,
 0.933452868705103,
 0.9169057577769292,
 0.863296725027191,
 0.8871546255777173,
 0.8850336384362296,
 0.9224024066197161,
 0.908047415300732,
 0.864715279073722,
 0.8383186655178901,
 0.8486545802774255,
 0.8865355337334098,
 0.8760041203923764,
 0.8109461543319932,
 0.823707733987434,
 0.8461956728386804,
 0.9022204207330559,
 0.8876446568881683,
 0.8229569063821714,
 0.8415666676384637,
 0.8525258660886917,
 0.8889432305292869,
 0.8786046910896893,
 0.8280638724382662,
 0.8432281034199491,
 0.8528314446628791,
 0.8926682099378198,
 

In [63]:
configurations = build_configurations(request_data, result)
configurations

[PackagingConfigurations(unitsPerPackage=45, foldingMethod='Method2', layout='LayoutC', supplier='SupplierA'),
 PackagingConfigurations(unitsPerPackage=45, foldingMethod='Method2', layout='LayoutC', supplier='SupplierA'),
 PackagingConfigurations(unitsPerPackage=45, foldingMethod='Method3', layout='LayoutC', supplier='SupplierB'),
 PackagingConfigurations(unitsPerPackage=45, foldingMethod='Method3', layout='LayoutC', supplier='SupplierC'),
 PackagingConfigurations(unitsPerPackage=5, foldingMethod='Method1', layout='LayoutC', supplier='SupplierD'),
 PackagingConfigurations(unitsPerPackage=5, foldingMethod='Method1', layout='LayoutC', supplier='SupplierE'),
 PackagingConfigurations(unitsPerPackage=45, foldingMethod='Method1', layout='LayoutC', supplier='SupplierF'),
 PackagingConfigurations(unitsPerPackage=5, foldingMethod='Method1', layout='LayoutC', supplier='SupplierG'),
 PackagingConfigurations(unitsPerPackage=5, foldingMethod='Method1', layout='LayoutC', supplier='SupplierH')]